In [1]:
with open("../../data/Pret_data/french_dataset.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [2]:
print(text[:100])

Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembr


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"%'(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz­ºÀÂÉÊÖàáâäçèéêëíîïòóôöøùûü
106


### tokenization 

In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[60, 61, 61, 1, 72, 60, 57, 70, 57]
hii there


In [5]:
import torch 
data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape,data.dtype)
data[:100]

torch.Size([1044950]) torch.int64


tensor([34, 57,  1, 56, 92, 55, 64, 53, 70, 57,  1, 70, 57, 68, 70, 61, 71, 57,
         1, 64, 53,  1, 71, 57, 71, 71, 61, 67, 66,  1, 56, 73,  1, 40, 53, 70,
        64, 57, 65, 57, 66, 72,  1, 57, 73, 70, 67, 68, 92, 57, 66,  1, 69, 73,
        61,  1, 53, 74, 53, 61, 72,  1, 92, 72, 92,  1, 61, 66, 72, 57, 70, 70,
        67, 65, 68, 73, 57,  1, 64, 57,  1, 74, 57, 66, 56, 70, 57, 56, 61,  1,
        13, 19,  1, 56, 92, 55, 57, 65, 54, 70])

### Split train and validation

In [6]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8
train_data[:block_size+1]

tensor([34, 57,  1, 56, 92, 55, 64, 53, 70])

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")
    

when input is tensor([34]) the target: 57
when input is tensor([34, 57]) the target: 1
when input is tensor([34, 57,  1]) the target: 56
when input is tensor([34, 57,  1, 56]) the target: 92
when input is tensor([34, 57,  1, 56, 92]) the target: 55
when input is tensor([34, 57,  1, 56, 92, 55]) the target: 64
when input is tensor([34, 57,  1, 56, 92, 55, 64]) the target: 53
when input is tensor([34, 57,  1, 56, 92, 55, 64, 53]) the target: 70


In [9]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 67,  66,  74,  61,  57,  66,  72,   1],
        [ 57,   1,  56,   5,   1,  67, 103,   1],
        [ 66,  72,   1,  68,  53,  71,   1,  56],
        [ 73,  71,  61,  67,  66,   1,  56,  57]])
targets:
torch.Size([4, 8])
tensor([[ 66,  74,  61,  57,  66,  72,   1,  56],
        [  1,  56,   5,   1,  67, 103,   1,  64],
        [ 72,   1,  68,  53,  71,   1,  56,  57],
        [ 71,  61,  67,  66,   1,  56,  57,   1]])
----
when input is [67] the target: 66
when input is [67, 66] the target: 74
when input is [67, 66, 74] the target: 61
when input is [67, 66, 74, 61] the target: 57
when input is [67, 66, 74, 61, 57] the target: 66
when input is [67, 66, 74, 61, 57, 66] the target: 72
when input is [67, 66, 74, 61, 57, 66, 72] the target: 1
when input is [67, 66, 74, 61, 57, 66, 72, 1] the target: 56
when input is [57] the target: 1
when input is [57, 1] the target: 56
when input is [57, 1, 56] the target: 5
when input is [57, 1, 56, 5] the target: 1
whe

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 106])
tensor(5.2726, grad_fn=<NllLossBackward0>)

-'Y)éYqÊLL)íf,à
3âD0ïQ,ÂdXW)0çjôóBû%øÂ%péç/Hp;ò5"räòBT6çíS
iág0Kg9òv1D Éf,Fh
Yºnbwnw'óîÉ;kcl'F!2UïÀr


In [11]:
# Training the Model 
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [12]:
batch_size = 32
for steps in range(2000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.1425392627716064


In [13]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


MZEotafn dje à A0aubM-'McW-DínôQJónorSIzNbupëq.alayUBû]XIDgÉ90ï,âl arlli k,ùère?ôtÖ%UAÀURuMè8ørenóRj


In [14]:
# Masked Self attention model 
import torch 
import torch.nn.functional as F

B,T,C = 4,8,2 
x = torch.randn(B,T,C)
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)  
wei = wei.masked_fill(tril == 0, float('-inf'))  # fill the upper triangular part with -inf
wei = F.softmax(wei, dim=-1)  # apply softmax to get the attention weights
out = wei @ x  


In [15]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
# implementing a single head of masked self attention 
import torch.nn as nn
head_size = 16
# x is private info to a token 
key = nn.Linear(C, head_size, bias=False) #key is what I have  
query = nn.Linear(C, head_size, bias=False) # query is what I am interested in
value = nn.Linear(C, head_size, bias=False) # if you find me interesting , here is what I will provide to you 

k = key(x)
q = query(x)
wei = q   @ k.transpose(-2, -1)   # scaled dot-product attention
wei = wei.masked_fill(tril == 0, float('-inf'))  # fill the upper triangular part with -inf
wei = F.softmax(wei, dim=-1)  # apply softmax to get the attention weights
out = wei @ value(x)
out.shape

torch.Size([4, 8, 16])

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
